# A notebook to perform QC on the PIPS slow T observations

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from mpl_toolkits.axes_grid1 import ImageGrid,make_axes_locatable,host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.polarimetric as dualpol
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
# from pyCRMtools.pycaps import arps_read
# from pyCRMtools.pycaps import pycaps_fields
# from pyCRMtools.pycaps import calvars_radar as radar
import pandas as pd
import xarray as xr
import xskillscore as xs
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.stats.mstats import zscore
from scipy.special import gamma as gammafunc
from scipy import ndimage
from metpy.plots import StationPlot
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
from cycler import cycler
import warnings
warnings.simplefilter('ignore')
%matplotlib notebook

In [ ]:
# plt.style.use('seaborn-v0_8-bright')

In [ ]:
# Read in the PIPS netcdf files for the case we want

# PIPS_input_base_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2022/PIPS_data/'
# PIPS_output_base_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2022/PIPS_data_for_EOL/'

# PIPS_base_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2023/PIPS_data/'
PIPS_base_dir = '/Users/dawson29/Dropbox/Projects/PERiLS/obsdata/2023/'
# PIPS_base_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2023/PIPS_data/'
# PIPS_base_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2022/PIPS_data/'

deployment_name = 'IOP2_030323' # '031623_mass_test' # 'IOP2_030323' # 'IOP5_040523' # 'IOP4_033123' # 'IOP3_032423' # 'IOP3_040522'
PIPS_input_dir = os.path.join(PIPS_base_dir, deployment_name, 'netcdf')
PIPS_output_dir = os.path.join(PIPS_base_dir, deployment_name, 'netcdf_thermoQC')

# deployment_name = '031623_mass_test'
# PIPS_input_dir = os.path.join(PIPS_base_dir, deployment_name, 'netcdf')
# PIPS_output_dir = os.path.join(PIPS_base_dir, deployment_name, 'netcdf_T_Td_QC_v110923')

# deployment_name = 'IOP2_033022' # '031623_mass_test' # 'IOP2_030323' # 'IOP5_040523' # 'IOP4_033123' # 'IOP3_032423' # 'IOP3_040522'
# PIPS_input_dir = os.path.join(PIPS_base_dir, deployment_name, 'netcdf')
# PIPS_output_dir = os.path.join(PIPS_base_dir, deployment_name, 'netcdf_T_Td_QC')

if not os.path.exists(PIPS_output_dir):
    os.makedirs(PIPS_output_dir)

# IOP1 2022
# PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS2B']
# IOP2 2022
# PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS3B']
# IOP3 2022
PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS2B', 'PIPS3A', 'PIPS3B']
# IOP3 2023
# PIPS_names = ['PIPS2A', 'PIPS3A']
# IOP2, IOP4 or IOP5 2023
# PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS2B', 'PIPS3A', 'PIPS3B']
parsivel_interval = 10
intervalstr = '10S'

parsivel_filenames = ['parsivel_combined_{}_{}_{:d}s.nc'.format(deployment_name, PIPS_name, parsivel_interval)
                      for PIPS_name in PIPS_names]
parsivel_filepaths = [os.path.join(PIPS_input_dir, parsivel_filename) for parsivel_filename in parsivel_filenames]
output_parsivel_filepaths = [os.path.join(PIPS_output_dir, parsivel_filename) 
                             for parsivel_filename in parsivel_filenames]
conv_filenames = ['conventional_raw_{}_{}.nc'.format(deployment_name, PIPS_name) for PIPS_name in PIPS_names]
conv_filepaths = [os.path.join(PIPS_input_dir, conv_filename) for conv_filename in conv_filenames]
output_conv_filepaths = [os.path.join(PIPS_output_dir, conv_filename) for conv_filename in conv_filenames]
parsivel_ds_dict = {}
conv_ds_dict = {}
for PIPS_name, parsivel_filepath, conv_filepath in zip(PIPS_names, parsivel_filepaths, conv_filepaths):
    try:
        parsivel_ds_dict[PIPS_name] = xr.load_dataset(parsivel_filepath)
    except:
        parsivel_ds_dict[PIPS_name] = None
    conv_ds_dict[PIPS_name] = xr.load_dataset(conv_filepath)

In [ ]:
# default_cycler = (cycler(color=['r', 'orange', 'y', 'g', 'b', 'purple']))

# plt.rc('lines', linewidth=4)
# plt.rc('axes', prop_cycle=default_cycler)

In [ ]:
PIPS_to_comp = PIPS_names

start_times = []
stop_times = []
for PIPS_name in PIPS_to_comp:
    start_times.append(conv_ds_dict[PIPS_name]['time'][0])
    stop_times.append(conv_ds_dict[PIPS_name]['time'][-1])
    
min_time = min(start_times)
max_time = max(stop_times)

print(min_time, max_time)

In [ ]:
time_start = min_time # '2023-03-17T8:30:00'
time_stop = max_time # '2023-03-17T9:30:00'

In [ ]:
# Remove any "corrected" thermodynamic variables that are already in the datasets
conv_ds_dict['PIPS1A']

vars_to_remove = ['fasttemp_corrected', 'slowtemp_corrected', 'RH_corrected', 'pressure_corrected',
                  'dewpoint_corrected', 'RH_derived_corrected', 'pt_corrected', 'qv_corrected', 
                  'rho_corrected']

for PIPS_name in PIPS_to_comp:
    try:
        conv_ds_dict[PIPS_name] = conv_ds_dict[PIPS_name].drop_vars(vars_to_remove)
    except ValueError:
        print("Vars already removed")
    try:
        parsivel_ds_dict[PIPS_name] = parsivel_ds_dict[PIPS_name].drop_vars(vars_to_remove)
    except ValueError:
        print("Vars already removed")

In [ ]:
# Create copies of the slowtemps for each PIPS which will be filled with corrected data

for PIPS_name in PIPS_to_comp:
    conv_ds_dict[PIPS_name]['slowtemp_corrected'] = conv_ds_dict[PIPS_name]['slowtemp'].copy()

In [ ]:
def prune_slowtemps(slowtemp, diff_tol=0.17, num_iter=10):
    """Attempts to clean up wonky slowtemp data (that has rapid upward fluctuations) by removing those values
       that deviate upward more than a threshold over a short period of time."""
    # difference fluctuating upward
    slowtemp_diff1 = slowtemp.diff("time", label='upper')
    # difference fluctuating back downward from the high point
    slowtemp_diff2 = slowtemp.diff("time", label='lower')
    
    slowtemp_pruned = slowtemp.copy()
    for i in range(num_iter):
        # The following prunes those points that deviate upward and drops the resulting NaN's
        slowtemp_pruned = slowtemp.where((slowtemp_diff1 < diff_tol) & (slowtemp_diff2 > -diff_tol), drop=True)
        # Recompute the diffs from the newly pruned timeseries. This will, with each iteration of the loop,
        # "eat away" at the erroneous upward deviations from either side.
        slowtemp_diff1 = slowtemp_pruned.diff("time", label='upper')
        slowtemp_diff2 = slowtemp_pruned.diff("time", label='lower')
    
    # Now go ahead and just take the "pruned" slowtemp data and fill in NaNs for all the dropped times.
    # Do this using xarray's "reindex_like" function
    # Unfortunately this leads to a *lot* of missing values
    slowtemp_new = slowtemp_pruned.reindex_like(slowtemp)
    
    return slowtemp_new

In [ ]:
# PIPS 3A and 3B both have some wonky issues with the slow temps where the temperature will periodically
# fluctuate upward by about half a degree, leading to a "ghost" temperature timeseries that lies above 
# the "true" temperature. Let's try to remove that ghost here:

for PIPS_name in ['PIPS3A', 'PIPS3B']:
    conv_ds = conv_ds_dict[PIPS_name]
    slowtemp = conv_ds['slowtemp']

    slowtemp_new = prune_slowtemps(slowtemp)

    fig, ax = plt.subplots()
    slowtemp_new.plot(ax=ax, label=f'slowtemp_new', ls='None', marker='o', ms=1., alpha=0.5)
    conv_ds_dict[PIPS_name]['slowtemp_corrected'] = slowtemp_new

In [ ]:
# Compute the differences between the fast and slow temps for each PIPS and then take the time average. 
# Then subtract the time-averaged diffs from the original slow temps and store them in a new
# "corrected" slow temp variable

diff_T_dict = {}

for PIPS_name in PIPS_to_comp:
    diff_T = conv_ds_dict[PIPS_name]['slowtemp_corrected'] - conv_ds_dict[PIPS_name]['fasttemp']
    diff_T_dict[PIPS_name] = diff_T

In [ ]:
# Plot the time series of the slow-fast temp differences
# Note there are a couple "spikes" in the difference that correspond to where there was a rapid drop in
# temperature. This spike reflects the different time constants of the fast and slow-response temperature
# sensors, but since the time period where this is happening is so short, we can still get by with a simple
# time average of all the diffs. If we really wanted to get fancy, we could probably exclude any period in 
# the average
# where the temperature is changing too much, but it probably wouldn't make much of a difference in this case.

fig, ax = plt.subplots()

for PIPS_name in PIPS_to_comp:
    diff_T = diff_T_dict[PIPS_name]
    diff_T.sel(time=slice(time_start, time_stop)).plot(ax=ax, label=f'{PIPS_name}_diff_T', 
                                                       ls='None', marker='o', ms=1., alpha=0.5)

ax.legend()

In [ ]:
# Compute the mean differences and then subtract from the original slowtemp, storing in a new corrected slowtemp

for PIPS_name in PIPS_to_comp:
    diff_T = diff_T_dict[PIPS_name]
    mean_diff_T = diff_T.mean().values
    print(PIPS_name, mean_diff_T)
    conv_ds_dict[PIPS_name]['slowtemp_corrected'] = conv_ds_dict[PIPS_name]['slowtemp_corrected'] - mean_diff_T
    # Add the value subtracted to the attributes so we know that we modified it
    conv_ds_dict[PIPS_name]['slowtemp_corrected'].attrs['bias_subtracted'] = mean_diff_T
    

In [ ]:
# Ok, now plot the bias-corrected slowtemps alongside the fasttemps
# Much better!


fig, ax = plt.subplots()

for PIPS_name in ['PIPS1B']:
    conv_ds = conv_ds_dict[PIPS_name]
    conv_ds['slowtemp_corrected'].sel(time=slice(time_start, time_stop)).plot(ax=ax, label=f'{PIPS_name}_slowT_corrected', 
                                                                              ls='None', marker='o', ms=1., alpha=0.5)    
# plt.gca().set_prop_cycle(None)

for PIPS_name in ['PIPS1B']:
    conv_ds = conv_ds_dict[PIPS_name]
    conv_ds['fasttemp'].sel(time=slice(time_start, time_stop)).plot(ax=ax, label=f'{PIPS_name}_fastT', 
                                                                    ls='None', marker='x', ms=5., alpha=0.75)
    
ax.legend()

In [ ]:
# Now, we need to recompute the dewpoint and RH_derived using the bias-corrected slowtemp from above

for PIPS_name in PIPS_to_comp:
    pressure = conv_ds_dict[PIPS_name]['pressure']
    slowtemp = conv_ds_dict[PIPS_name]['slowtemp_corrected']
    fasttemp = conv_ds_dict[PIPS_name]['fasttemp']
    RH = conv_ds_dict[PIPS_name]['RH']
    dewpoint = thermo.calTdfromRH(pressure * 100., slowtemp + 273.15, RH / 100.) - 273.15
#     dewpoint.sel(time=slice(time_start, time_stop)).plot(ax=ax, label=f'{PIPS_name}_dewpoint', 
#                                                          ls='None', marker='o', ms=1., alpha=0.5)
    RH_derived = thermo.calRH(pressure * 100., fasttemp + 273.15, dewpoint + 273.15) * 100.
    
    conv_ds_dict[PIPS_name]['dewpoint_corrected'] = conv_ds_dict[PIPS_name]['dewpoint'].copy()
    conv_ds_dict[PIPS_name]['dewpoint_corrected'].data = dewpoint
    
    conv_ds_dict[PIPS_name]['RH_derived_corrected'] = conv_ds_dict[PIPS_name]['RH_derived'].copy()
    conv_ds_dict[PIPS_name]['RH_derived_corrected'].data = RH_derived

In [ ]:
# Plot the timeseries of RH_derived_corrected
fig, ax = plt.subplots()

for PIPS_name in PIPS_to_comp:
    conv_ds = conv_ds_dict[PIPS_name]
    conv_ds['RH_derived_corrected'].sel(time=slice(time_start, time_stop)).plot(ax=ax, label=f'{PIPS_name}_RH_derived_corrected', 
                                                              ls='None', marker='o', ms=1., alpha=0.5)    
# plt.gca().set_prop_cycle(None)

# for PIPS_name in PIPS_to_comp:
#     conv_ds = conv_ds_dict[PIPS_name]
#     conv_ds['RH_derived_corrected'].sel(time=slice(time_start, time_stop)).plot(ax=ax, label=f'{PIPS_name}_RH_derived_cor', 
#                                                                     ls='None', marker='x', ms=5., alpha=0.75)
    
ax.legend()

In [ ]:
# Plot the timeseries of dewpoint_corrected
fig, ax = plt.subplots()

for PIPS_name in PIPS_to_comp:
    conv_ds = conv_ds_dict[PIPS_name]
    conv_ds['dewpoint_corrected'].sel(time=slice(time_start, time_stop)).plot(ax=ax, label=f'{PIPS_name}_dewpoint'), 

ax.legend()

In [ ]:
# Recompute some of the derived thermodynamic parameters (pt, qv, rho) using the new corrected vars

for PIPS_name in PIPS_to_comp:
    conv_ds = conv_ds_dict[PIPS_name]
    conv_ds = pips.calc_thermo(conv_ds, p_var='pressure', T_var='fasttemp', 
                               RH_var='RH_derived_corrected', suffix='_corrected')

In [ ]:
# Plot the timeseries of pt_corrected to check
fig, ax = plt.subplots()

for PIPS_name in PIPS_to_comp:
    conv_ds = conv_ds_dict[PIPS_name]
    conv_ds['pt_corrected'].sel(time=slice(time_start, time_stop)).plot(ax=ax, label=f'{PIPS_name}_pt'), 

ax.legend()

In [ ]:
# Now resample the corrected timeseries to the parsivel times

corrected_varnames = ['slowtemp_corrected', 'dewpoint_corrected', 'RH_derived_corrected', 'pt_corrected', 
                      'qv_corrected', 'rho_corrected']

for PIPS_name in PIPS_to_comp:
    PSD_datetimes = pips.get_PSD_datetimes(parsivel_ds_dict[PIPS_name]['VD_matrix'])
    sec_offset = PSD_datetimes[0].second
    print(sec_offset)
    offset_str = pips.get_interval_str(sec_offset)
    
    for corrected_varname in corrected_varnames:
        corrected_var = conv_ds_dict[PIPS_name][corrected_varname]
        new_var = corrected_var.resample(time=intervalstr, label='right', closed='right', 
                                         offset=offset_str).mean()
        
        parsivel_ds_dict[PIPS_name][corrected_varname] = new_var
        parsivel_ds_dict[PIPS_name][corrected_varname].attrs = conv_ds_dict[PIPS_name][corrected_varname].attrs

In [ ]:
fig, ax = plt.subplots()

for PIPS_name in PIPS_to_comp:
    parsivel_ds_dict[PIPS_name]['pt_corrected'].plot(ax=ax, label=f'{PIPS_name}_pt_corrected', 
                                                             ls='None', marker='o', ms=1., alpha=0.75)
#     parsivel_ds_dict[PIPS_name]['RH_derived'].plot(ax=ax, label=f'{PIPS_name}_RH_derived', 
#                                                    ls='None', marker='x', ms=5., alpha=0.5)
    
ax.legend()

In [ ]:
# Now save to new output directory
for PIPS_name, output_parsivel_filepath, output_conv_filepath in zip(PIPS_names, 
                                                                     output_parsivel_filepaths, 
                                                                     output_conv_filepaths):
    if PIPS_name in PIPS_to_comp:
        print(PIPS_name)
        
        
        print("Saving {}".format(output_parsivel_filepath))
        parsivel_ds_dict[PIPS_name].to_netcdf(output_parsivel_filepath)
        print("Saving {}".format(output_conv_filepath))
        conv_ds_dict[PIPS_name].to_netcdf(output_conv_filepath)